scrapping data 


In [1]:
import requests
import pandas as pd
from bs4 import BeautifulSoup
import time
import urllib.request
import os

def scrape_data():
    SerieA_italy = "https://fbref.com/en/comps/11/Serie-A-Stats"
    LaLiga_spain = "https://fbref.com/en/comps/12/La-Liga-Stats"

    urls = [SerieA_italy, LaLiga_spain]
    
    for url in urls:
        standings_url = url
        splitted_url = standings_url.split('/')
        last_split = splitted_url[-1]
        
        # Cập nhật danh sách năm từ 2026 đến 2011
        years = list(range(2026, 2015, -1))
        
        if url == SerieA_italy:
            last_split = last_split + "-Italy"
        
        all_matches = []

        for year in years:
            print("\n Scrapping Data for Year", year)
            
            data = requests.get(standings_url)
            soup = BeautifulSoup(data.text, "html.parser")

            standings_table = soup.select("table.stats_table")[0]

            links = standings_table.find_all('a')
            href = [link.get('href') for link in links]
            href_squad = [link for link in href if '/squads/' in link]
            team_urls = [f'https://fbref.com{link}' for link in href_squad]

            try:
                previous_season = soup.select("a.prev")[0].get("href")
                standings_url = f'https://fbref.com{previous_season}'
            except IndexError:
                previous_season = ""
                standings_url = ""

            for team_url in team_urls:
                team_name = team_url.split('/')[-1].replace('-Stats', '').replace('-', ' ')

                data = requests.get(team_url)
                matches = pd.read_html(data.text, match='Scores & Fixtures')[0]

                soup = BeautifulSoup(data.text, "html.parser")
                links = soup.find_all('a')
                href = [link.get('href') for link in links]
                href_shooting = [link for link in href if link and 'all_comps/shooting' in link]
                data = requests.get(f'https://fbref.com/{href_shooting[0]}')

                shooting = pd.read_html(data.text, match='Shooting')[0]
                shooting.columns = shooting.columns.droplevel()

                try:
                    team_data = pd.concat([matches, shooting[shooting.columns.difference(matches.columns)]], axis=1)
                except ValueError:
                    print("A column is missing")
                    continue
                
                print("\n > Team Appending:", team_name)
                
                team_data['Year'] = year
                team_data['Team'] = team_name

                all_matches.append(team_data)
                time.sleep(10)

        match_df = pd.concat(all_matches)
        match_df.columns = [c.lower() for c in match_df.columns]
        
        last_split = last_split.replace('-', '_')
        match_df.to_csv("../Datasets/Scrapped Datasets/" + last_split + ".csv")

if __name__ == "__main__":
    scrape_data()

KeyboardInterrupt: 